### 1. 주주가치 측면에서의 잉여현금흐름(Free Cash Flow to Equity)

  : 채권자에게 귀속되는 부분을 제외한 주주귀속부분을 추출하기 위해 이자비용, 부채상환금을 차감한다. 

  : 현금흐름은 재무적인 관점으로 회계적 관점에서 쓰여진 재무제표에 없는 변수들을 입력하여야 한다.

* 영업이익 - **이자비용** = EBT
* EBT(Earning Before Tax) - 법인세 비용 = EAT(Earning After Tax)
* EAT - 순운전자본변동분(NWC) + 비현금비용(감가상각비) - 자본적지출(CAPEX) - **부채상환** = FCFE

* 영업활동을 영위하기 위해 자산성 있는 항목을 현금을 유출시켜 늘리는 것으로 당기손익에 영향을 미치지 않는 지출을 자본적 지출(= 영업기반의 기업투자지출)이라고 한다. 

  예) 토지, 건물 기타 부동산, 기계, 차량, 컴퓨터
  
  유형자산, 취득원가에 가산한다. 자본적 지출에 의한 감가상각비 일부는 손익계산서상 반영되지만, 현금흐름이 반영되지 않기 때문에 별도로 산출하여야 할 것이다. 

### 2. 가중평균자본비용(Weighted Average Cost of Capital)

  : 기업의 재무실적 평가 시 사업에 투자한 자산의 이익률이 자본비용(투자자의 요구/기대수익률)을 얼마만큼 상회했는지를 기준으로 한다. 
  
* 무위험수익률: 국고채 3년물 금리 [한국은행 경제통계시스템](https://ecos.bok.or.kr/#/) ecos.bok.or.kr/#/Short/a744bc ecos.bok.or.kr/#/Short/7ce443
* 시장수익률: 

  (1) S&P500수익률에서 한국 국가조정을 한 수익률, 국가별 리스크 지표인 CDS 스프레드(채권에서 주식 조정하기 위해 Market Equity Index 표준편차 대비 BAML Emerging Public Bond Index 표준편차 비율을 이용하여 산출)

  (2) 유가증권시장 상장기업의 경우 KOSPI지수

* 동종업계 베타

### 3. Discounted Cash Flow, 기본적 분석 
* $ EV = \Sigma_{n=1}^{T} \frac{FCF_n}{(1+WACC)^n} + TV $ (기업가치)
* $ TV = \frac{\frac{FCF_T+1}{WACC-g}}{(1+WACC)^T} $ (기업의 내재가치)
* $ WACC = \frac{D}{V}K_d(1-t) + \frac{E}{V}K_e $



출처 : 
* [DCF계산법, DCF를 통한 EV 계산법](https://valuefactory.tistory.com/m/859)
* [파이썬으로 DCF 구현하기](https://hiyihiyii.tistory.com/m/63)
* https://site.financialmodelingprep.com/developer/docs/

In [1]:
import pandas as pd
  # 데이터 처리
import numpy as np
  # 선형대수, 행렬 연산 등
import requests
import json, traceback
from bs4 import BeautifulSoup as soup 
  # 크롤링할 때 url을 통해 웹페이지 html 문서 (문자열) 요청
import re
  # 정규표현식
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
  # 날짜형 타입 처리

``` 
세후영업이익 - 순운전자본 변동 + 감가상각비(비현금비용) - 자본적지출 = 잉여현금흐름(FCFF)
```

In [62]:
class DiscountCashFlow1():

  def __init__(self, html_stock_code: str, ticker = None,
               Income_statement = None, Balance_sheet = None, Cash_flow = None):
    
    self.html_stock_code = html_stock_code
    self._ticker = ticker
    self.Income_statement = Income_statement
    self.Balance_sheet = Balance_sheet
    self.Cash_flow = Cash_flow

    self._매출액 = []
    self._매출원가 = []
    self._판매관리비 = []
    self._법인세율 = 0
    self._operate_cashflow = []
    
    self._capex_list = []
    self._depreciation_list = []
    self._amortization_list = []
    self._NWC = []
    self._dNWC = []
    self._FCFF = []


# getStockCode() : 상장주식종목 코드번호를 html을 통해 데이터프레임 형태로 불러오는 함수
  def getStockCode(self) -> pd.DataFrame:
  
  # 1. 상장지수 회사명 및 종목코드 데이터를 불러오기
    KOSPI_code_list = pd.read_html(self.html_stock_code + "?method=download&marketType=stockMkt")[0]
    KOSDAQ_code_list = pd.read_html(self.html_stock_code + "?method=download&marketType=kosdaqMkt")[0]
    KOSPI_code_list = KOSPI_code_list[["회사명", "종목코드"]]
    KOSDAQ_code_list = KOSDAQ_code_list[["회사명", "종목코드"]]
      # 각 시장별 회사명, 종목코드만 추출
    KOSPI_code_list["종목코드"] = KOSPI_code_list["종목코드"].apply(lambda x: "0"*(6 - len(str(x))) + str(x))
    KOSDAQ_code_list["종목코드"] = KOSDAQ_code_list["종목코드"].apply(lambda x: "0"*(6 - len(str(x))) + str(x))
      # 주식종목코드는 int 타입(연속형)으로 6자리에 맞추기 위한 0 추가하는 익명함수(lambda)

  # 2. KOSPI, KOSDAQ 상장주식종목에서 리츠와 ETF를 제거
  # 문자열 정규표현식에서 "리츠"와 "스팩"을 필터링하여 해당 행 번호를 추출하여 데이터프레임에서 삭제
    KOSPI_code_list.drop(
        KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
    KOSPI_code_list.drop(
        KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(리츠)") == True].index.tolist(), axis = 0, inplace = True)
    KOSDAQ_code_list.drop(
        KOSDAQ_code_list[KOSDAQ_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
    KOSDAQ_code_list.drop(
        KOSDAQ_code_list[KOSDAQ_code_list["회사명"].str.contains(r"(리츠)") == True].index.tolist(), axis = 0, inplace = True)
      # [str.contains UserWarning] This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
    return KOSPI_code_list, KOSDAQ_code_list


# loadFinancialData() : 재무 데이터 크롤링하여 손익계산서, 대차대조표, 현금흐름표 계정별 데이터프레임 캡슐화하기
  def loadFinancialData(self): # ticker_name: str):
    # 자료 출처 : FnGuide, Naver Finanace, Yahoo Finance
    self.ticker = str(input("종목코드: "))
    # self.ticker = ticker_name
    url_fnguide = '''https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'''.format(self.ticker)
    table_fin_data = pd.read_html(url_fnguide, encoding = "utf-8")

    self.Income_statement = table_fin_data[0]
      # 손익계산서
    self.Balance_sheet = table_fin_data[2]
      # 대차대조표
    self.Cash_flow = table_fin_data[4]
      # 현금흐름표

    print(self.Income_statement)
    print(self.Balance_sheet)
    print(self.Cash_flow)


# IncomeSheet() 손익계산서상 계정과목
# 매출액, 매출원가, 판매관리비 -> 영업현금흐름과 법인세율 반환
  def IncomeSheet(self):
    IS = self.Income_statement.loc[
        [0, 1, 3], ["IFRS(연결)","2019/12","2020/12","2021/12","2022/12"]
    ]
    self._매출액 = list(IS.loc[0])
    self._매출원가 = list(IS.loc[1])
    self._판매관리비 = list(IS.loc[3])
      # [TypeError] cannot do slice indexing on Index with these indexers [1] of type int
    self._매출액.pop(0)
    self._매출원가.pop(0)
    self._판매관리비.pop(0)


    for index in range(4):
      if self._매출액[index] > 200:
        # 3,000억 원 초과면 법인세율 22%(과거 25%)
        # 200억 원 초과 3,000억 원 이하이면 법인세율 22%
        self._법인세율 = 0.22
      elif 2 < self._매출액[index] <= 200:
        # 5억 ~ 200억 원, 2 ~ 5억 원, 2억 원 이하이면 법인세율 20%
        self._법인세율 = 0.20
      else:
        pass

      self._operate_cashflow.append(
          (1 - self._법인세율)
          *(self._매출액[index] - self._매출원가[index] - self._판매관리비[index])
      )
        # 매출액 - 매출원가 = 매출총이익
        # 매출총이익 - 판매관리비 = 영업이익
        # 영업이익*(1-법인세율) = 영업현금흐름
      self._법인세율


# CashFlow() 현금흐름표상 계정과목
# 자본적지출, 유형자산상각비(감가상각비), 무형자산상각비를 반환
  def CashFlow(self):
    url_fnguide = '''https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'''.format(self.ticker)
    response_fin_data = requests.get(url_fnguide)

    if response_fin_data.status_code == 200:
      html = response_fin_data.text
      object = soup(html, "html.parser")
        # [UnboundLocalError] local variable 'soup' referenced before assignment
      capex = object.select_one("#divCashY > table > tbody > tr:nth-child(108)")
      depreciation = object.select_one("#divCashY > table > tbody > tr:nth-child(9)")
        # [TypeError] select_one() missing 1 required positional argument: 'selector'
      amortization = object.select_one("#divCashY > table > tbody > tr:nth-child(10)")
    else:
      print(response_fin_data.status_code)
      
    capex = capex.get_text()
      # 자본적지출(영업 기반의 기업투자지출)
    depreciation = depreciation.get_text()
      # 유형자산상각비 : 감가상각비(시간의 흐름에 따른 유형자산의 가치 감소)
    amortization = amortization.get_text()
      # 무형자산상각비 : 영업권, 저작권, 개발비 등의 무형자산을 회계적으로 배분한 비용
      # 직접 해당 자산의 계정에서 금액을 차감

    regex = re.compile("\d+\,?\d+")
      # 정규표현식 : 모든 숫자 추출 (\d+)/(\d+)
    self._capex_list = regex.findall(capex)
    self._depreciation_list = regex.findall(depreciation)
    self._amortization_list = regex.findall(amortization)
      # 문자열 중 정규표현식의 패턴과 일치하는 모든 부분을 찾아 캡슐화 

    for index in range(4):
      self._capex_list[index] = int(self._capex_list[index].replace(",", ""))
      self._depreciation_list[index] = int(self._depreciation_list[index].replace(",", ""))
      self._amortization_list[index] = int(self._amortization_list[index].replace(",", ""))
    
    # self._capex_list.pop()
    # self._depreciation_list.pop()
    # self._amortization_list.pop()
    print(self._capex_list, self._depreciation_list, self._amortization_list)
    

# BalancSheet() 재무상태표상 계정과목
# 매출채권, 재고자산, 기타유동자산, 매입채무, 유동부채 -> 순운전자본 변동분 반환
  def BalanceSheet(self):
    url_fnguide = '''https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'''.format(self.ticker)
    response_fin_data = requests.get(url_fnguide)
    
    if response_fin_data.status_code == 200:
      # 웹페이지의 html 문서 데이터 요청에 응답하는데 성공하면 HTTP 상태코드 200 (404 Not found, 403 Forbidden, 500 Internal Server Error)
      html_fin_data = response_fin_data.text
      object_fin_data = soup(html_fin_data, "html.parser")

      # 매출채권 및 기타유동채권
      account_receivable_request = object_fin_data.select_one("#divDaechaY > table > tbody > tr:nth-child(6)").get_text()
      # 재고자산
      inventory_assets_request = object_fin_data.select_one("#divDaechaY > table > tbody > tr:nth-child(3)").get_text()
      # 기타유동자산(선급금 등)
      other_current_assets_request = object_fin_data.select_one("#divDaechaY > table > tbody > tr:nth-child(11)").get_text()
      
      # 매입채무 및 기타유동채무
      account_assets_request = object_fin_data.select_one("#divDaechaY > table > tbody > tr:nth-child(35)").get_text()
      # 유동부채
      other_current_liabilities_request = object_fin_data.select_one("#divDaechaY > table > tbody > tr:nth-child(42)").get_text()
    else: 
      print(response_fin_data.status_code)
    # print(html_fin_data)

    regex = re.compile("\d+\,?\d+")
      # 하나 이상의 숫자인 정규식 객체를 반환
    
    # [TypeError] expected string or bytes-like object
    account_receivable = regex.findall(account_receivable_request)
    inventory_assets = regex.findall(inventory_assets_request)
    other_current_assets = regex.findall(other_current_assets_request)
    account_payable = regex.findall(account_assets_request)
    other_current_liabilities = regex.findall(other_current_liabilities_request)

    for index in range(len(account_receivable)):
      account_receivable[index] = int(account_receivable[index].replace(",", ""))
      inventory_assets[index] = int(inventory_assets[index].replace(",", ""))
      other_current_assets[index] = int(other_current_assets[index].replace(",", ""))
      account_payable[index] = int(account_payable[index].replace(",", ""))
      other_current_liabilities[index] = int(other_current_liabilities[index].replace(",", ""))
      # 순운전자본변동(Net Working Capital) = 유동자산과 유동부채의 차이, 유동자산 / 총자본
      self._NWC.append(
          (account_receivable[index] + inventory_assets[index] + other_current_assets[index])
           - (account_payable[index] + other_current_liabilities[index]))
        # 영업활동을 하면서 회계상 수익 비용 인식 대비 실제 현금이 어느수준으로 유입되는지 확인하는 척도
        # 해석: NWC 증가하면 유동부채가 더 증가했다는 것, 현금이 회계상 인식보다 덜 유입 혹은 더 유출되었다는 의미

    # self._NWC.pop()

    for index in range(len(self._operate_cashflow)):
      # 영업현금흐름 리스트의 길이를 인덱스로 설정
      if index < 3:
        self._dNWC.append(
            self._NWC[index+1] - self._NWC[index]
        )
        # 전년 대비 당기 순운전자본 NWC의 변동분 d를 계산하여 리스트에 원소로 추가
    self._dNWC.insert(0, 0)
    return self._dNWC


# FreeCashFlowtotheFirm()
  def FreeCashFlowtotheFirm(self):
    for index in range(4):
      self._FCFF.append(
         self._operate_cashflow[index]
         + self._depreciation_list[index]
         + self._amortization_list[index]
         - self._capex_list[index]
         # IndexError: list index out of range
         - self._dNWC[index]
     )
    
    return self._FCFF
  

In [79]:
DCF1 = DiscountCashFlow1(    
    html_stock_code = "https://kind.krx.co.kr/corpgeneral/corpList.do",
)

In [80]:
KOSPI_code, KOSDAQ_code = DCF1.getStockCode()

<ipython-input-62-738f46eb4d87>:42: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
<ipython-input-62-738f46eb4d87>:44: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  KOSPI_code_list[KOSPI_code_list["회사명"].str.contains(r"(리츠)") == True].index.tolist(), axis = 0, inplace = True)
<ipython-input-62-738f46eb4d87>:46: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  KOSDAQ_code_list[KOSDAQ_code_list["회사명"].str.contains(r"(스팩)") == True].index.tolist(), axis = 0, inplace = True)
<ipython-input-62-738f46eb4d87>:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the group

In [81]:
KOSPI_code[KOSPI_code["회사명"] == "LG에너지솔루션"]

,회사명,종목코드
424,LG에너지솔루션,373220


In [82]:
DCF1.loadFinancialData()
  # input: "373220" (단위: 억 원)

종목코드: "373220"
                              IFRS(연결)    2019/12    2020/12    2021/12  \
0                                  매출액  2304009.0  2368070.0  2796048.0   
1                                 매출원가  1472395.0  1444883.0  1664113.0   
2                                매출총이익   831613.0   923187.0  1131935.0   
3                판매비와관리비계산에 참여한 계정 펼치기   553928.0   563248.0   615596.0   
4                                 영업이익   277685.0   359939.0   516339.0   
5                           영업이익(발표기준)   277685.0   359939.0   516339.0   
6                   금융수익계산에 참여한 계정 펼치기   101616.0   122676.0    85432.0   
7                   금융원가계산에 참여한 계정 펼치기    82749.0   113181.0    77046.0   
8                   기타수익계산에 참여한 계정 펼치기    17787.0    13841.0    22057.0   
9                   기타비용계산에 참여한 계정 펼치기    14147.0    24889.0    20560.0   
10  종속기업,공동지배기업및관계기업관련손익계산에 참여한 계정 펼치기     4130.0     5065.0     7296.0   
11                            세전계속사업이익   304322.0   363451.0   533518.0   
12        

view-source:https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode={}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701

In [83]:
DCF1.IncomeSheet()
  # Error의 원인 ['매출액', 2304009.0, 2368070.0, 2796048.0, 3022314.0]

In [84]:
DCF1.CashFlow()

[253678, 375920, 471221, 494304] [265738, 271157, 312852, 359521] [30238, 32199, 29622, 31556]


In [85]:
DCF1.BalanceSheet()

[0, -61017, 96185, 81947]

In [86]:
DCF1.FreeCashFlowtotheFirm() 

[258893.08000000002, 269205.4199999999, 177812.42000000004, 153163.47999999998]

In [72]:
class DiscountCashFlow2():
  # def __init__(self):
  
  # def weightedAverageCapitalCost(self):